# Data Science Capstone Project - Week 3 Assignment

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download Dependencies</a>

2. <a href="#item2">Scrape and Parse Table</a>

3. <a href="#item3">Dataframe after Screenscrape (First Submission)</a>

4. <a href="#item4">Gather Latitude and Longitude (Second Submission)</a>

5. <a href="#item5">Explore Toronto Neighborhoods (Third Submission)</a>

</font>
</div>

<a id='item1'></a>

## Download Dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [2]:
# Install Beautiful Soup
#from bs4 import BeautifulSoup

## Beautiful Soup not used - opted to use pandas read instead

In [3]:
## Beautiful Soup not used - opted to use pandas read instead
#Get Toronto Data: Postal Codes starting with 'M'
#import urllib2 #enable ability to read URLs
#from urllib.request import urlopen #enable ability to read URLs

#postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#page = urlopen(postal_url).read()
#soup = BeautifulSoup(page, 'html.parser')

#postal_frame = []
#postal_table =  soup.find('table', {'class': 'wikitable sortable'})

#rows = postal_table.findAll('tr')
#for row in rows:
#    cols=row.findAll('td')
#    cols=[ele.text.strip() for ele in cols]
#    postal_frame.append([ele for ele in cols if ele])

##postal_frame.shape
##postal_table

<a id='item2'></a>

## Scrape and Parse Table using pandas

In [4]:
#Get Toronto Data: Postal Codes starting with 'M' using pandas.read
#import urllib2 #enable ability to read URLs

import requests
postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(postal_url)

postal_frame = pd.read_html(page.content,attrs = {'class': 'wikitable sortable'})[0]


postal_frame.shape


(288, 3)

In [5]:
postal_frame.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# Process dataframe
#Remove Borough 'Not assigned' rows
postal_frame_clean = postal_frame.drop(postal_frame[postal_frame.Borough == 'Not assigned'].index)

#Change Neighborhood to match Borough if Neighborhood = 'Not assigned'
postal_frame_clean['Neighbourhood'][postal_frame_clean['Neighbourhood'] == 'Not assigned'] = postal_frame_clean['Borough']

#Combine multiple neighborhoods from the same Burough into one row
postal_frame_clean = postal_frame_clean.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

#Rename Postcode Field to PostalCode
postal_frame_clean.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

#postal_frame_clean.head()

<a id='item3'></a>

## Answer to Question 3 (First Submission)

In [7]:

postal_frame_clean.shape


(103, 3)

<a id='item4'></a>

## Gather Latitude and Longitude for each Postal code in our dataframe (Submission 2)

In [8]:
#Geocoder didn't work so using the file that was provided in class

#import geocoder # import geocoder
#postal_code = 'M5G'
## initialize your variable to None
#lat_lng_coords = None

## loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

In [9]:
#Read File provided with Coordinates for each postal code

coords_file = r'Geospatial_Coordinates.csv'
coords = pd.read_csv(coords_file)

coords.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [10]:
coords.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
coords.shape

(103, 3)

In [12]:
#add lat/longitude to the postal_frame_clean frame

toronto_df_full = pd.merge(postal_frame_clean, coords, on = 'PostalCode' )

#GetSubset of Data = Borough contains 'Toronto'
toronto_df = toronto_df_full[toronto_df_full['Borough'].str.contains('Toronto')]
#Debug - uncomment below to focus on just one Burough = East Toronto
#toronto_df = toronto_df_full[toronto_df_full['Borough']=='East Toronto'] #Test to limit dataframe to generate map
toronto_df.shape
#toronto_df.head()


(38, 5)

<a id='item5'></a>

# Analysis of Toronto Neighborhoods (Submission 3)

### Import modules needed for data exploration

In [13]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



In [14]:
#!conda install -c conda-forge folium=0.10.0 #Already installed
#!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

In [15]:
#Toronto Latitude/Longitude
latitude = 43.653908
longitude = -79.384293

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=7)
#map_toronto

html = """<h1> this is a popup</h1><br>"""

iframe = folium.IFrame(html=html, width=500, height=300)
popup=folium.Popup(iframe, max_width=2650)


#folium.Marker(location=[43.68, -79.29]).add_to(map_toronto)
#folium.Marker(location=[43.67635739999999, -79.2930312]).add_to(map_toronto)
#folium.Marker(location=[43.67635739999999, -79.2930312], popup="The Beaches").add_to(map_toronto)
#folium.Marker([43.6795571, -79.352188], popup='<i>Danforth West</i>').add_to(map_toronto)
#folium.Marker([43.6689985,-79.31557159999998], popup='<i>Beaches West</i>').add_to(map_toronto)


# add markers to map
for lat, lng, borough, neighborhood in list(zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood'])):
    #print("Neighborhood: ", neighborhood, "/Borough: ", borough, "/Lat: ", lat, "/Long: ", lng)
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
# Foursquare Credentials /To be Hidden/Deleted from posted version
CLIENT_ID = '' #redacted # your Foursquare ID
CLIENT_SECRET = '' #Redacted # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

## Explore DataNeighbourhood

In [17]:
# Function to find list of nearby venues near a given list of neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    #print("test round done")
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [18]:
# type your answer here

#for Hood in manhattan_data
#    manhattan_venues = manhattan_venues.append(getNearbyVenues(names=Hood['Neighborhood'], latitudes=Hood['Latitude'], longitudes['Longitude']))


toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )



The Beaches 43.67635739999999 -79.2930312
The Danforth West,Riverdale 43.6795571 -79.352188
The Beaches West,India Bazaar 43.6689985 -79.31557159999998
Studio District 43.6595255 -79.340923
Lawrence Park 43.7280205 -79.3887901
Davisville North 43.7127511 -79.3901975
North Toronto West 43.7153834 -79.40567840000001
Davisville 43.7043244 -79.3887901
Moore Park,Summerhill East 43.6895743 -79.38315990000001
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West 43.68641229999999 -79.4000493
Rosedale 43.6795626 -79.37752940000001
Cabbagetown,St. James Town 43.667967 -79.3676753
Church and Wellesley 43.6658599 -79.38315990000001
Harbourfront,Regent Park 43.6542599 -79.3606359
Ryerson,Garden District 43.6571618 -79.37893709999999
St. James Town 43.6514939 -79.3754179
Berczy Park 43.644770799999996 -79.3733064
Central Bay Street 43.6579524 -79.3873826
Adelaide,King,Richmond 43.65057120000001 -79.3845675
Harbourfront East,Toronto Islands,Union Station 43.6408157 -79.38175229999999
Design

#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [21]:
toronto_onehot.shape

(1711, 235)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.071429,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.000000,0.000000,0.000000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.010753,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,...,0.010753,0.010753,0.000000,0.000000,0.00,0.000000,0.010753,0.010753,0.000000,0.010753


In [23]:
toronto_grouped.shape

(38, 235)

#### Print each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2       Steakhouse  0.04
3              Bar  0.04
4  Thai Restaurant  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2      Steakhouse  0.04
3          Bakery  0.04
4  Farmers Market  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3      Restaurant  0.05
4     Music Venue  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1                Garden  0.06
2      Recording Studio  0.06
3            Smoke Shop  0.06
4  Fast Food Restaurant  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0   Airport Lounge  0.14
1  Airport Service  0.14
2  Harbor / Marina  0.07
3    

## Segment Data

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Restaurant,American Restaurant,Burger Joint,Hotel,Thai Restaurant,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Farmers Market,Cheese Shop,Steakhouse,Café,Italian Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Bar,Stadium,Bakery
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Farmers Market,Butcher,Restaurant,Burrito Place,Auto Workshop,Fast Food Restaurant,Spa,Recording Studio
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Boat or Ferry,Boutique,Bar,Coffee Shop,Airport Terminal,Sculpture Garden,Airport Gate,Airport


<a id='item4'></a>

## Cluster Data

Run *k*-means to cluster the neighborhood into 5 clusters.

In [27]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df
#Rename Neighbourhood Field to Neighborhood - prob should have done this higher up, but wamted the earlier submission dataset to match exactly
toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)



# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

C:\Users\david\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Wings Joint,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Café,Sports Bar,Indian Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pizza Place,Sushi Restaurant,Fish & Chips Shop,Steakhouse,Ice Cream Shop,Liquor Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Diner,Pizza Place,Park,Middle Eastern Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Dim Sum Restaurant,Bus Line,Swim School,Wings Joint,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


Finally, let's visualize the resulting clusters

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Wings Joint,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Café,Sports Bar,Indian Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Pizza Place,Sushi Restaurant,Fish & Chips Shop,Steakhouse,Ice Cream Shop,Liquor Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Diner,Pizza Place,Park,Middle Eastern Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Dim Sum Restaurant,Bus Line,Swim School,Wings Joint,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Examine Clusters

#### Cluster 1

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,0,Health Food Store,Pub,Trail,Wings Joint,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
41,East Toronto,"The Danforth West,Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Yoga Studio,Café,Sports Bar,Indian Restaurant
42,East Toronto,"The Beaches West,India Bazaar",0,Sandwich Place,Pizza Place,Sushi Restaurant,Fish & Chips Shop,Steakhouse,Ice Cream Shop,Liquor Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant
43,East Toronto,Studio District,0,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gym / Fitness Center,Diner,Pizza Place,Park,Middle Eastern Restaurant
45,Central Toronto,Davisville North,0,Gym,Hotel,Clothing Store,Dog Run,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gay Bar,Eastern European Restaurant
46,Central Toronto,North Toronto West,0,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Diner,Salon / Barbershop,Restaurant,Park,Mexican Restaurant,Metro Station,Cosmetics Shop
47,Central Toronto,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Seafood Restaurant
49,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0,Coffee Shop,Pub,American Restaurant,Liquor Store,Restaurant,Sports Bar,Bagel Shop,Supermarket,Fried Chicken Joint,Sushi Restaurant
51,Downtown Toronto,"Cabbagetown,St. James Town",0,Coffee Shop,Bakery,Market,Park,Italian Restaurant,Pizza Place,Restaurant,Café,Pub,Gastropub
52,Downtown Toronto,Church and Wellesley,0,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub,Café,Gastropub,Hotel,Fast Food Restaurant


#### Cluster 2

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,Roselawn,1,Garden,Wings Joint,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 3

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,"Moore Park,Summerhill East",2,Gym,Restaurant,Playground,Trail,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 4

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + [2]  + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,Rosedale,3,Park,Playground,Trail,Building,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
64,Central Toronto,"Forest Hill North,Forest Hill West",3,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,Lawrence Park,4,Park,Dim Sum Restaurant,Bus Line,Swim School,Wings Joint,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
